In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [31]:
mnist_dataset = input_data.read_data_sets('MNIST_dataset/', one_hot=True)

Extracting MNIST_dataset/train-images-idx3-ubyte.gz
Extracting MNIST_dataset/train-labels-idx1-ubyte.gz
Extracting MNIST_dataset/t10k-images-idx3-ubyte.gz
Extracting MNIST_dataset/t10k-labels-idx1-ubyte.gz


In [4]:
# weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
# bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
# convolution
def conv2d(x, W):
    # x.shape = [batch, height, width, nb_channels]
    # W.shape = [height, width, channels in, channels out]
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [7]:
# pooling
def max_pool(x):
    # x.shape = [batch, height, width, nb_channels]
    # ksize and strides: batch, h, w, nb_channels
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [13]:
# convolution layer
def conv_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [18]:
# fully connected layer
def fully_connected(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [10]:
# placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [11]:
# layers
x_image = tf.reshape(x, [-1, 28, 28, 1]) # it reshapes 784 to 28 by 28 with 1 layer for a channel

In [14]:
conv_1 = conv_layer(x_image, [5, 5, 1, 32])
pool_1 = max_pool(conv_1)

In [16]:
conv_2 = conv_layer(pool_1, [5, 5, 32, 64])
pool_2 = max_pool(conv_2)

In [19]:
pool_2_flat = tf.reshape(pool_2, [-1, 7*7*64]) # applying pooling two times decreased the shape four times
full_layer_one = tf.nn.relu(fully_connected(pool_2_flat, 1024))

In [21]:
# dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [22]:
y_pred = fully_connected(full_one_dropout, 10)

In [23]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [25]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [26]:
init = tf.global_variables_initializer()

In [33]:
steps = 5000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist_dataset.train.next_batch(50)
        sess.run(train, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})
        
        if i%100 == 0:
            print('On step: {}'.format(i))
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            print('Accuracy: {}'.format(sess.run(
                acc, 
                feed_dict={
                    x: mnist_dataset.test.images,
                    y_true: mnist_dataset.test.labels,
                    hold_prob: 1.0
                }
            )))

On step: 0
Accuracy: 0.15950000286102295
On step: 100
Accuracy: 0.9383999705314636


KeyboardInterrupt: 